In [1]:
import json
from pathlib import Path

from bids.layout import BIDSLayout
from bids.modeling import BIDSStatsModelsGraph

In [2]:
root = 'data/ds003425_no_ses'
db_path = 'data/ds003425_no_ses/dbcache'
reset_database = True
spec_path = 'model_specs/ds003425_spec_no_ses.json'

In [3]:
layout = BIDSLayout(
    root=root,
    database_path=db_path,
    reset_database=reset_database,
)

In [4]:
spec = json.loads(Path(spec_path).read_text())

In [5]:
graph = BIDSStatsModelsGraph(layout, spec)
graph.load_collections(scan_length=250)  # scan_length is in seconds

### Original `BIDSVariableCollection`

We can take a look at the original variables available for a single subject, prior to running the node (i.e. applying any transformations)

Note, my model spec doesn't query the derivatives at all, so I think that's why they don't show up as options for regressors?  Not sure


In [6]:
root_node = graph.root_node
colls = root_node.get_collections()
first_sub = colls[0]

# This is what there is to work with in the transformations

In [7]:
first_sub.to_df(entities=False)

,duration,onset,other_type,trial_type
0,6.5,3.544157,four,4
1,6.5,23.571931,one,1
2,6.5,43.599812,two,2
3,6.5,63.627572,three,3
4,6.5,83.655411,four,4
5,6.5,103.683288,three,3
6,6.5,123.711044,five,5
7,6.5,143.738938,five,5
8,6.5,163.766710,two,2
9,6.5,183.794586,four,4


### Variables that can be used as Input for the first transformation

In [8]:
first_sub.variables

{'other_type': <SparseRunVariable(name='other_type', source='events')>,
 'trial_type': <SparseRunVariable(name='trial_type', source='events')>}

### There are currently no dense variables (defined for each TR).  I assume the derivatives would show up here.

In [9]:
first_sub.get_dense_variables()

[]

### Running the node (and applying transformations)

These are the `Transformations` that wil be applied:

In [10]:
graph.model['nodes'][0]['transformations']

{'transformer': 'pybids-transforms-v1',
 'instructions': [{'name': 'Factor', 'input': ['other_type']},
  {'name': 'Convolve',
   'input': ['other_type.one', 'other_type.two'],
   'model': 'spm'}]}

In [11]:
root_node.group_by

['run', 'subject']

In [12]:
for col in root_node.get_collections():
    print(col.entities)

{'FlipAngle': np.int64(90), 'AcquisitionTime': '14:54:36.447500', 'SequenceName': '_epfid2d1_64', 'AcquisitionNumber': np.int64(1), 'datatype': 'func', 'RepetitionTime': np.int64(2), 'ScanningSequence': 'EP', 'ConversionSoftwareVersion': 'v1.0.20170724 GCC4.4.7', 'EffectiveEchoSpacing': np.float64(0.000356093), 'TaskName': 'learning', 'InstitutionName': 'USC', 'task': 'learning', 'BodyPartExamined': 'BRAIN', 'PhaseEncodingDirection': 'i', 'BandwidthPerPixelPhaseEncode': np.float64(43.879), 'SoftwareVersions': 'syngo_MR_D13D', 'ProcedureStepDescription': 'Mara_Steve', 'EchoTrainLength': np.int64(43), 'TotalReadoutTime': np.float64(0.0299118), 'SequenceVariant': 'SK_OSP', 'extension': '.nii.gz', 'ImageType': '["ORIGINAL", "PRIMARY", "M", "ND", "NORM", "MOCO", "MOSAIC"]', 'EchoTime': np.float64(0.025), 'InstitutionAddress': 'McClintock_3520A_Los_Angeles_CA_US_90089', 'DeviceSerialNumber': '67025', 'SliceTiming': '[0.99, 0, 1.0425, 0.0525, 1.095, 0.105, 1.1475, 0.1575, 1.2, 0.21, 1.25, 0.2

In [13]:
type(root_node)

bids.modeling.statsmodels.BIDSStatsModelsNode

In [14]:
root_node.get_collections

<bound method BIDSStatsModelsNode.get_collections of <BIDSStatsModelsNode(level=run, name=Within-run)>>

In [15]:
dir(root_node)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_build_groups',
 '_collections',
 '_group_data',
 'add_child',
 'add_collections',
 'add_parent',
 'children',
 'contrasts',
 'dummy_contrasts',
 'get_collections',
 'group_by',
 'level',
 'model',
 'name',
 'parents',
 'run',
 'transformations']

In [16]:
outputs = root_node.run(
    group_by=root_node.group_by,
    force_dense=False,
    transformation_history=True,
)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()